In [ ]:
#@title Default title text
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [1]:
import pandas as pd
import numpy as np

def to_month_id(year, month):
  return (year - 1996) * 12 + month - 6

n_features = 12
lookback = 3
chosen_stocks = ["NVDA"] # "AMZN", "MSFT", "IBM", "INTC", "QCOM", "NVDA", "IBM", "ADBE", "EBAY", "CSCO"

In [2]:
dfFeature = pd.read_csv("./lstm_2004_12.csv")
# dfFeature.loc[dfFeature["return"] == 0, "return"] = 1
dfFeature = dfFeature[dfFeature.QAId.isin(chosen_stocks)]
features = dfFeature.drop(columns=['month_id', 'QAId']).as_matrix()
print(features.shape)

(157, 12)


In [3]:
from sklearn.preprocessing import MinMaxScaler
normalizedFeatures = MinMaxScaler().fit_transform(features) \
                                   .reshape(157, len(chosen_stocks), n_features) \
                                   .reshape(157, len(chosen_stocks) * n_features)
print(normalizedFeatures.shape)

(157, 12)


In [4]:
dfTarget = pd.read_csv("./return_2004_40.csv")
dfTarget["return"] = np.sign(dfTarget["return"])
# dfTarget.loc[dfTarget["return"] == 0, "return"] = 1
dfTarget = dfTarget[dfTarget.QAId.isin(chosen_stocks)]
dfTarget = dfTarget[dfTarget.month_id >= (to_month_id(2004, 1) + lookback)]
targets = MinMaxScaler().fit_transform(dfTarget.drop(columns=['month_id', 'QAId']).as_matrix())
y = targets.reshape(157 - lookback + 1, len(chosen_stocks))
print(y.shape)

(155, 1)


In [5]:
from keras.preprocessing.sequence import TimeseriesGenerator

splittedFeature = normalizedFeatures
print(splittedFeature.shape)

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(157, 12)


In [6]:
from keras.preprocessing.sequence import TimeseriesGenerator

x = np.zeros((157 - lookback + 1, lookback, n_features * len(chosen_stocks)))
y_mock = np.zeros((157, len(chosen_stocks)))

i = 0
for train, test in TimeseriesGenerator(splittedFeature, y_mock, length=lookback, batch_size=1):
  if i > 157 - lookback:
    break
  x[i] = train[0]
  i = i + 1

In [7]:
from sklearn.model_selection import PredefinedSplit

test_months = 26
end_point = 157 - lookback + 1
split_point = 157 - lookback + 1 - test_months

x_train = x[0:split_point - 48].reshape(split_point - 48, lookback, n_features * len(chosen_stocks))
y_train = y[0:split_point - 48].reshape(split_point - 48, len(chosen_stocks))
x_validate = x[split_point - 36:split_point].reshape(36, lookback, n_features * len(chosen_stocks))
y_validate = y[split_point - 36:split_point].reshape(36, len(chosen_stocks))
x_test = x[split_point:end_point].reshape(test_months, lookback, n_features * len(chosen_stocks))
y_test = y[split_point:end_point].reshape(test_months, len(chosen_stocks))

print(x_train.shape)
print(y_train.shape)
print(x_validate.shape)
print(y_validate.shape)
print(x_test.shape)
print(y_test.shape)

# ps = PredefinedSplit(np.append(np.negative(np.ones(60 * 465)), np.zeros(24 * 465))).split(x_train)

# for train_ids, test_ids in ps:
#   print(str(train_ids) + ", " + str(test_ids))

(81, 3, 12)
(81, 1)
(36, 3, 12)
(36, 1)
(26, 3, 12)
(26, 1)


In [8]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Flatten, CuDNNLSTM
from keras.regularizers import l1_l2, l2
from keras.optimizers import RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

np.random.seed(4103)

def create_model(*param):
    model = Sequential()
    model.add(LSTM(units=256, input_shape=(lookback, n_features * len(chosen_stocks)), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=128, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(units=64, return_sequences=False))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(32, activation="relu"))
    model.add(Dense(len(chosen_stocks), activation="relu"))
    model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001, decay=0.0), metrics=['accuracy'], *param)
    return model

In [9]:
batch_size=30

model = create_model()
train_score = model.fit(x_train, y_train, validation_data=(x_validate, y_validate), batch_size=batch_size, epochs=200)
print(train_score)

Train on 81 samples, validate on 36 samples
Epoch 1/200
81/81 [==============================] - 2s 25ms/step - loss: 9.0431 - acc: 0.3827 - val_loss: 5.5018 - val_acc: 0.3889
Epoch 2/200
81/81 [==============================] - 0s 755us/step - loss: 4.8471 - acc: 0.3827 - val_loss: 3.0194 - val_acc: 0.3889
Epoch 3/200
81/81 [==============================] - 0s 753us/step - loss: 2.9937 - acc: 0.3827 - val_loss: 2.6897 - val_acc: 0.3889
Epoch 4/200
81/81 [==============================] - 0s 735us/step - loss: 2.7125 - acc: 0.3827 - val_loss: 2.5158 - val_acc: 0.3889
Epoch 5/200
81/81 [==============================] - 0s 705us/step - loss: 2.5490 - acc: 0.3827 - val_loss: 2.3943 - val_acc: 0.3889
Epoch 6/200
81/81 [==============================] - 0s 752us/step - loss: 2.4183 - acc: 0.3827 - val_loss: 2.2993 - val_acc: 0.3889
Epoch 7/200
81/81 [==============================] - 0s 738us/step - loss: 2.3325 - acc: 0.3827 - val_loss: 2.2205 - val_acc: 0.3889
Epoch 8/200
81/81 [=======

Epoch 62/200
81/81 [==============================] - 0s 758us/step - loss: 0.7256 - acc: 0.3951 - val_loss: 0.7094 - val_acc: 0.4722
Epoch 63/200
81/81 [==============================] - 0s 718us/step - loss: 0.7201 - acc: 0.4444 - val_loss: 0.7056 - val_acc: 0.5000
Epoch 64/200
81/81 [==============================] - 0s 685us/step - loss: 0.7435 - acc: 0.3827 - val_loss: 0.7040 - val_acc: 0.5000
Epoch 65/200
81/81 [==============================] - 0s 709us/step - loss: 0.6877 - acc: 0.5679 - val_loss: 0.7044 - val_acc: 0.5000
Epoch 66/200
81/81 [==============================] - 0s 710us/step - loss: 0.6864 - acc: 0.5432 - val_loss: 0.7059 - val_acc: 0.5556
Epoch 67/200
81/81 [==============================] - 0s 753us/step - loss: 0.6785 - acc: 0.5679 - val_loss: 0.7085 - val_acc: 0.6111
Epoch 68/200
81/81 [==============================] - 0s 781us/step - loss: 0.6927 - acc: 0.5679 - val_loss: 0.7116 - val_acc: 0.6111
Epoch 69/200
81/81 [==============================] - 0s 788us

Epoch 123/200
81/81 [==============================] - 0s 762us/step - loss: 0.6706 - acc: 0.5802 - val_loss: 0.7229 - val_acc: 0.6111
Epoch 124/200
81/81 [==============================] - 0s 777us/step - loss: 0.6725 - acc: 0.5802 - val_loss: 0.7239 - val_acc: 0.6111
Epoch 125/200
81/81 [==============================] - 0s 717us/step - loss: 0.6701 - acc: 0.6049 - val_loss: 0.7250 - val_acc: 0.6111
Epoch 126/200
81/81 [==============================] - 0s 736us/step - loss: 0.6473 - acc: 0.6543 - val_loss: 0.7266 - val_acc: 0.6111
Epoch 127/200
81/81 [==============================] - 0s 732us/step - loss: 0.6471 - acc: 0.6543 - val_loss: 0.7283 - val_acc: 0.6111
Epoch 128/200
81/81 [==============================] - 0s 707us/step - loss: 0.6666 - acc: 0.6049 - val_loss: 0.7281 - val_acc: 0.6111
Epoch 129/200
81/81 [==============================] - 0s 709us/step - loss: 0.6544 - acc: 0.6173 - val_loss: 0.7277 - val_acc: 0.6111
Epoch 130/200
81/81 [==============================] - 

Epoch 184/200
81/81 [==============================] - 0s 713us/step - loss: 0.6634 - acc: 0.6173 - val_loss: 0.7243 - val_acc: 0.6111
Epoch 185/200
81/81 [==============================] - 0s 703us/step - loss: 0.6515 - acc: 0.6173 - val_loss: 0.7264 - val_acc: 0.6111
Epoch 186/200
81/81 [==============================] - 0s 680us/step - loss: 0.6708 - acc: 0.5679 - val_loss: 0.7276 - val_acc: 0.6111
Epoch 187/200
81/81 [==============================] - 0s 711us/step - loss: 0.6475 - acc: 0.6173 - val_loss: 0.7270 - val_acc: 0.6111
Epoch 188/200
81/81 [==============================] - 0s 748us/step - loss: 0.6623 - acc: 0.5802 - val_loss: 0.7267 - val_acc: 0.6111
Epoch 189/200
81/81 [==============================] - 0s 699us/step - loss: 0.6629 - acc: 0.6296 - val_loss: 0.7251 - val_acc: 0.6111
Epoch 190/200
81/81 [==============================] - 0s 676us/step - loss: 0.6588 - acc: 0.5926 - val_loss: 0.7256 - val_acc: 0.6111
Epoch 191/200
81/81 [==============================] - 

In [10]:
import matplotlib.pyplot as plt

test_score = model.evaluate(x_test, y_test, batch_size=batch_size)
for i, mn in enumerate(model.metrics_names):
  print(mn + ": " + str(test_score[i]))

plt.plot(train_score.history["loss"])
plt.plot(train_score.history["val_loss"])
plt.title("Loss")
plt.xlabel("Epoch")
plt.ylabel("Metrics")
plt.legend(["Training", "Validation"], loc="upper left")
plt.show()

26/26 [==============================] - 0s 168us/step
loss: 0.571491003036499
acc: 0.7692307829856873


In [11]:
from sklearn.metrics import precision_score, recall_score, classification_report

def threshold_tester(result, target, a, b):
  for t in np.arange(a, b + 0.05, 0.05):
    print("thresholder: " + str(t))
    print(classification_report(target, (result >= t).astype(int)))

In [12]:
result = model.predict(x_test, batch_size=batch_size)
print(result)
threshold_tester(result, y_test, 0.1, 0.9)

[[0.7330345 ]
 [0.7097299 ]
 [0.6890042 ]
 [0.7087363 ]
 [0.6843182 ]
 [0.6923036 ]
 [0.70800185]
 [0.6822752 ]
 [0.72044843]
 [0.7433708 ]
 [0.7166606 ]
 [0.72552884]
 [0.65152824]
 [0.6184411 ]
 [0.58702195]
 [0.63106406]
 [0.67911893]
 [0.6690615 ]
 [0.68958217]
 [0.621814  ]
 [0.6595629 ]
 [0.6245035 ]
 [0.5663051 ]
 [0.626034  ]
 [0.63364345]
 [0.62671244]]
thresholder: 0.1
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         6
        1.0       0.77      1.00      0.87        20

avg / total       0.59      0.77      0.67        26

thresholder: 0.15000000000000002
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         6
        1.0       0.77      1.00      0.87        20

avg / total       0.59      0.77      0.67        26

thresholder: 0.20000000000000004
             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         6
        1.0    

/Users/billykwok/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
import math

buy_list = []
sell_list = []

midpt = (max(map(lambda x: x[0], result)) + min(map(lambda x: x[0], result))) / 2

for i, r in enumerate(result):
  buy_or_sell = 1 if r.item() > midpt * 1.05 else (-1 if r.item() < midpt * 0.95 else 0)
  if r.item() > midpt * 1.05:
    buy_list.append({'month_id': i + 223, 'QAId': chosen_stocks[0]})
  if r.item() < midpt * 0.95:
    sell_list.append({'month_id': i + 223, 'QAId': chosen_stocks[0]})
  print(str(y_test[i].item()) + ": " + str(buy_or_sell))
  if (math.fabs(buy_or_sell - y_test[i].item()) == 2) or (buy_or_sell - y_test[i].item() == 1):
    print("Hey")
print(buy_list)
print(sell_list)

1.0: 1
0.0: 1
Hey
1.0: 1
1.0: 1
0.0: 0
0.0: 1
Hey
1.0: 1
1.0: 0
1.0: 1
1.0: 1
1.0: 1
0.0: 1
Hey
1.0: 0
1.0: -1
Hey
0.0: -1
1.0: 0
1.0: 0
1.0: 0
1.0: 1
1.0: -1
Hey
1.0: 0
1.0: 0
1.0: -1
Hey
1.0: 0
0.0: 0
1.0: 0
[{'month_id': 223, 'QAId': 'NVDA'}, {'month_id': 224, 'QAId': 'NVDA'}, {'month_id': 225, 'QAId': 'NVDA'}, {'month_id': 226, 'QAId': 'NVDA'}, {'month_id': 228, 'QAId': 'NVDA'}, {'month_id': 229, 'QAId': 'NVDA'}, {'month_id': 231, 'QAId': 'NVDA'}, {'month_id': 232, 'QAId': 'NVDA'}, {'month_id': 233, 'QAId': 'NVDA'}, {'month_id': 234, 'QAId': 'NVDA'}, {'month_id': 241, 'QAId': 'NVDA'}]
[{'month_id': 236, 'QAId': 'NVDA'}, {'month_id': 237, 'QAId': 'NVDA'}, {'month_id': 242, 'QAId': 'NVDA'}, {'month_id': 245, 'QAId': 'NVDA'}]


In [16]:
dfBuy = pd.DataFrame(buy_list, columns=["month_id", "QAId"])
dfSell = pd.DataFrame(sell_list, columns=["month_id", "QAId"])
dfBuy.head(5)
dfSell.head(5)

,month_id,QAId
0,236,NVDA
1,237,NVDA
2,242,NVDA
3,245,NVDA


In [17]:
# from google.colab import files

filename_base = "_".join([stock.lower() for stock in chosen_stocks])
# filename_model = "./" + filename_base + "_model.h5"
# filename_weights = "./" + filename_base + "_weights.h5"
filename_output_buy = "./" + filename_base + "_output_buy.h5"
filename_output_sell = "./" + filename_base + "_output_sell.h5"

# model.save(filename_model)
# model.save_weights(filename_weights)
dfBuy.to_csv(filename_output_buy, index=False)
dfSell.to_csv(filename_output_sell, index=False)

# files.download(filename_model)
# files.download(filename_weights)
# files.download(filename_output_buy)
# files.download(filename_output_sell)